<a href="https://colab.research.google.com/github/andreagolfari/TensorFlow_notes/blob/main/00_tensorflow_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fundamentals of Tensorflow





* Introduction to tensors
* Getting information from tensors
* Tensors in NumPy
* Speeding regular Python functions using @tf.function
* Using GPUs or TPUs

## Introduction to Tensors


In [4]:
# Import Tensorflow
import tensorflow as tf
print(tf.__version__)

2.8.2


In [5]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

Output says constant is a tensor with empty shape, data type integer 32 bits of precision, and in numpy its value is 7.

In [6]:
# Check the number of dimensions of a tensor

scalar.ndim

0

In [7]:
# Create a vector

vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [8]:
vector.ndim

1

In [9]:
# Create a matrix

matrix = tf.constant([[8, 31],
                     [12, 7]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 8, 31],
       [12,  7]], dtype=int32)>

In [10]:
# Create a matrix specifying data type

another_matrix = tf.constant([[8., 31.],
                              [12., 7.],
                              [64., 69.]], dtype = tf.float16)
print(another_matrix)
print(another_matrix.ndim)

tf.Tensor(
[[ 8. 31.]
 [12.  7.]
 [64. 69.]], shape=(3, 2), dtype=float16)
2


In [11]:
# Create a tensor
tensor = tf.constant([[[8, 31, 3],
                       [12, 7, 4]],
                      [[64, 69, 4],
                       [36, 7, 98]],
                      [[13, 59, 88],
                       [49, 42, 6]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 8, 31,  3],
        [12,  7,  4]],

       [[64, 69,  4],
        [36,  7, 98]],

       [[13, 59, 88],
        [49, 42,  6]]], dtype=int32)>

In [12]:
tensor.ndim

3

## Creating tensors with `tf.Variable`

In [13]:
# Create the same tensor by using tf.Variable
changeable_tensor = tf.Variable([10, 10])
unchangeable_tensor = tf.constant([10, 10])

unchangeable_tensor, changeable_tensor

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>,
 <tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10, 10], dtype=int32)>)

In [14]:
# Make a change in a variable tensor
changeable_tensor[0].assign(7)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Make a change in a constant tensor with 

`unchangeable_tensor[0].assign(7)` 

will fail

## Creating random tensors
Random tensors are tensors of arbitrary size which contain random numbers.

In [15]:
## Creating random tensors with reproduceable values

random_1 = tf.random.Generator.from_seed(11249) # set seed
random_1 = random_1.normal(shape=(3,2))

random_2 = tf.random.Generator.from_seed(11249) # set seed
random_2 = random_2.normal(shape=(3,2))

# Checking that they are equal
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 0.97845185,  0.42823294],
        [-0.59423876, -1.5069418 ],
        [ 0.17741887, -0.03283609]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 0.97845185,  0.42823294],
        [-0.59423876, -1.5069418 ],
        [ 0.17741887, -0.03283609]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

## Shuffle the order of elements in a tensor 

In [16]:
# Shuffle a tensor
not_shuffled = tf.constant([[12, 8],
                            [36, 69],
                            [4, 88]])

# tf.random.shuffle shuffles along the first dimension
tf.random.shuffle(not_shuffled)


# setting a seed
tf.random.shuffle(not_shuffled, seed=11249)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[12,  8],
       [36, 69],
       [ 4, 88]], dtype=int32)>

In [17]:
# Setting an operation level seed produces each time a different shuffle
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 4, 88],
       [36, 69],
       [12,  8]], dtype=int32)>

In [18]:
# Setting a global seed produces same shuffle every time
tf.random.set_seed(11249)
tf.random.shuffle(not_shuffled, seed=5)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[36, 69],
       [ 4, 88],
       [12,  8]], dtype=int32)>

To obtain the shuffled tensors in the same order for reproducibility one needs to set both the global seed and the operation level seed.

> Rule for reproducibility: set global seed and specify operation seed when shuffling tensors

## Other ways to create tensors

In [19]:
# Create tensor of all ones
tf.ones(shape = [4, 6])

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [20]:
# Create tensor of all zeros
tf.zeros(shape = [4, 6])

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)>

### Turn Numpy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU with much faster computing. Otherwise they are very similar data structures.

In [21]:
# Create Numpy array
import numpy as np
numpy_A = np.arange(1, 25, dtype = np.int32)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [22]:
# Turn the numpy array into a tensor
A = tf.constant(numpy_A)
A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [23]:
# Turn the numpy array into a tensor specifying shape
B = tf.constant(numpy_A, shape = (2,3,4))
B

<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]], dtype=int32)>

## Getting information from tensors
* Shape
* Rank
* Axis or Dimension
* Size

In [24]:
# Access tensor shape
B.shape

TensorShape([2, 3, 4])

In [24]:
# Access tensor rank
B.ndim